<a href="https://colab.research.google.com/github/SushmithaUW/641-Assignments/blob/master/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="https://colab.research.google.com/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Welcome to Colaboratory!</h1>


Colaboratory is a free Jupyter notebook environment that requires no setup and runs entirely in the cloud.

With Colaboratory you can write and execute code, save and share your analyses, and access powerful computing resources, all for free from your browser.

In [0]:
#@title Introducing Colaboratory { display-mode: "form" }
#@markdown This 3-minute video gives an overview of the key features of Colaboratory:
from IPython.display import YouTubeVideo
YouTubeVideo('inN8seMm7UI', width=600, height=400)

## Getting Started

The document you are reading is a  [Jupyter notebook](https://jupyter.org/), hosted in Colaboratory. It is not a static page, but an interactive environment that lets you write and execute code in Python and other languages.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

In [0]:
seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

86400

To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter".

All cells modify the same global state, so variables that you define by executing a cell can be used in other cells:

In [0]:
seconds_in_a_week = 7 * seconds_in_a_day
seconds_in_a_week

604800

For more information about working with Colaboratory notebooks, see [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb).


## More Resources

Learn how to make the most of Python, Jupyter, Colaboratory, and related tools with these resources:

### Working with Notebooks in Colaboratory
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas](/notebooks/mlcc/intro_to_pandas.ipynb)
- [Tensorflow concepts](/notebooks/mlcc/tensorflow_programming_concepts.ipynb)
- [First steps with TensorFlow](/notebooks/mlcc/first_steps_with_tensor_flow.ipynb)
- [Intro to neural nets](/notebooks/mlcc/intro_to_neural_nets.ipynb)
- [Intro to sparse data and embeddings](/notebooks/mlcc/intro_to_sparse_data_and_embeddings.ipynb)

### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

## Machine Learning Examples: Seedbank

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out the [Seedbank](https://research.google.com/seedbank/) project.

A few featured examples:

- [Neural Style Transfer](https://research.google.com/seedbank/seed/neural_style_transfer_with_tfkeras): Use deep learning to transfer style between images.
- [EZ NSynth](https://research.google.com/seedbank/seed/ez_nsynth): Synthesize audio with WaveNet auto-encoders.
- [Fashion MNIST with Keras and TPUs](https://research.google.com/seedbank/seed/fashion_mnist_with_keras_and_tpus): Classify fashion-related images with deep learning.
- [DeepDream](https://research.google.com/seedbank/seed/deepdream): Produce DeepDream images from your own photos.
- [Convolutional VAE](https://research.google.com/seedbank/seed/convolutional_vae): Create a generative model of handwritten digits.

In [0]:

import gensim
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, BatchNormalization, Activation, Bidirectional
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
import csv
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import nltk
from sklearn.utils import shuffle

import csv
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import nltk
from sklearn.utils import shuffle
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors


In [0]:
MAX_SENT_LEN = 30
MAX_VOCAB_SIZE = 30000
LSTM_DIM = 128
EMBEDDING_DIM = 100
BATCH_SIZE = 32
N_EPOCHS = 5

In [0]:
#p = pd.read_csv('Tokenized_with_stopword_pos.csv', sep ='\n', names=['sentiment'])
#n = pd.read_csv('Tokenized_with_stopword_neg.csv', sep ='\n', names=['sentiment'])

p_train = pd.read_csv('train_no_stopword_pos.csv', sep ='\n', names=['sentiment'])
n_train = pd.read_csv('train_no_stopword_neg.csv', sep ='\n', names=['sentiment'])
p_train['label'] = 1
n_train['label'] = 0
p_train = shuffle (p_train.append(n_train))

p_val = pd.read_csv('val_no_stopword_pos.csv', sep ='\n', names=['sentiment'])
n_val = pd.read_csv('val_no_stopword_neg.csv', sep ='\n', names=['sentiment'])
p_val['label'] = 1
n_val['label'] = 0
p_val = shuffle (p_val.append(n_val))




In [0]:
for i in range(len(p_train.sentiment.values)):
    p_train.sentiment.values[i] = (p_train.sentiment.values[i].replace('\'', '').replace('[','').replace(',','').replace(']','')).split(' ')
    p_train.sentiment.values[i].remove('.')
    
for i in range(len(p_val.sentiment.values)):
    p_val.sentiment.values[i] = (p_val.sentiment.values[i].replace('\'', '').replace('[','').replace(',','').replace(']','')).split(' ')
    p_val.sentiment.values[i].remove('.')


In [90]:
p = p_train.append(p_val)

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts([' '.join(seq[:MAX_SENT_LEN]) for seq in p.sentiment.values])
print("Number of words in vocabulary:", len(tokenizer.word_index))



Number of words in vocabulary: 28564


In [0]:
X_train = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in p_train.sentiment.values])
X_train = pad_sequences(X_train, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

y_train = p_train.label.values

X_val = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in p_val.sentiment.values])
X_val = pad_sequences(X_val, maxlen=MAX_SENT_LEN, padding='post', truncating='post')

y_val = p_val.label.values



#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.1)
#X_train.shape

In [92]:
embeddings = KeyedVectors.load('mymodel.kv', mmap='r')
embeddings_matrix = np.random.uniform(-0.05, 0.05, size=(len(tokenizer.word_index)+1, EMBEDDING_DIM)) # +1 is because the matrix indices start with 0



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [107]:
model = Sequential()
#model = Sequential()



#model.add(Dense(1, activation='sigmoid'))

model.add(Embedding(input_dim=len(tokenizer.word_index)+1,
                          output_dim=EMBEDDING_DIM,
                          weights = [embeddings_matrix], trainable=True, name='word_embedding_layer', 
                          mask_zero=True))
model.add(Dense(32, activation='relu', name = 'input_layer'))

model.add(LSTM(LSTM_DIM, return_sequences=False, name='lstm_layer'))

model.add(Dense(1, activation='softmax', name='output_layer'))
#model.add(Dense(1, activation='softmax'))


model.summary()

#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_embedding_layer (Embedd (None, None, 100)         2856500   
_________________________________________________________________
input_layer (Dense)          (None, None, 32)          3232      
_________________________________________________________________
lstm_layer (LSTM)            (None, 128)               82432     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 2,942,293
Trainable params: 2,942,293
Non-trainable params: 0
_________________________________________________________________


In [108]:
model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=5,
          validation_data=(X_val, y_val))

Train on 64000 samples, validate on 8000 samples
Epoch 1/5
64000/64000 [==============================] - 132s 2ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/5
  256/64000 [..............................] - ETA: 2:02 - loss: 7.9712 - acc: 0.5000

KeyboardInterrupt: ignored